In [1]:
import QuantLib as ql
from collections import namedtuple
import math
from pandas import DataFrame
import pandas as pd
import numpy as np
from datetime import datetime


导入校准得到的参数
* `using_zerobond_vol`: 使用ZeroBond数据校准得到的volatility
* `using_swaption_vol`: 使用swaption数据校准得到的volatility
* a_calibrated: mean reversion

In [2]:
#volatility (62,)
#导入使用zerobong校准得到的数据
using_zerobond_vol = np.array(pd.read_excel('zerobond_sigma.xlsx', sheet_name='Sheet1'), dtype=float)
using_zerobond_vol = np.round(using_zerobond_vol, 8)

using_swaption_vol = np.array(pd.read_excel('listmin_sigma_as_initial.xlsx', sheet_name='Sheet1'), dtype=float)
using_swaption_vol = np.round(using_swaption_vol, 8)

a_calibrated = np.array(pd.read_excel('a_calibrated.xlsx', sheet_name='Sheet1').iloc[0:62,0], dtype=float) 

#zero rate
ZeroRate = np.array(pd.read_excel('ZeroRate.xlsx', sheet_name='Sheet1').iloc[0:62,0:601], dtype=float) * 0.01
#ZeroRate.shape=(62, 601)

#Maturity（star）、Tenor（length）序列
Maturity = np.tile(np.array([1,2,3,4,5,6,7,8,10]), 8)
Tenor = np.repeat([1,2,3,4,5,10,15,20], 9)
    

In [3]:
def create_swaption_helpers(data, index, term_structure, engine):
    swaptions = []
    fixed_leg_tenor = ql.Period(3, ql.Months) 
    fixed_leg_daycounter = ql.Actual360()
    floating_leg_daycounter = ql.Actual360()# 浮动和固定利率的天数计算方式
    for d in data:
        vol_handle = ql.QuoteHandle(ql.SimpleQuote(d.volatility))
        helper = ql.SwaptionHelper(ql.Period(int(d.start), ql.Years),ql.Period(int(d.length), ql.Years),vol_handle,index,fixed_leg_tenor,fixed_leg_daycounter,floating_leg_daycounter,term_structure)
        helper.setPricingEngine(engine)
        swaptions.append(helper)
    return swaptions

In [4]:
def calibration_report(swaptions, data):
    columns = ["Model Price", "Market Price", "Implied Vol", "Market Vol","Rel Error Price", "Rel Error Vols"]
    report_data = []
    cum_err = 0.0
    cum_err2 = 0.0
    for i, s in enumerate(swaptions):
        model_price = s.modelValue()
        market_vol = data[i].volatility
        black_price = s.blackPrice(market_vol)
        rel_error = model_price/black_price - 1.0
        implied_vol = s.impliedVolatility(model_price,1e-5, 50, 0.0, 0.50)
        rel_error2 = implied_vol/market_vol-1.0
        cum_err += rel_error*rel_error
        cum_err2 += rel_error2*rel_error2
        report_data.append((model_price, black_price, implied_vol,market_vol, rel_error, rel_error2))
    print("Cumulative Error Price: %7.5f" % math.sqrt(cum_err))
    print("Cumulative Error Vols : %7.5f" % math.sqrt(cum_err2))
    return DataFrame(report_data, columns=columns,index=['']*len(report_data))


In [5]:
Quantlib_price_swaption=np.full((62,72),1.0)  #62个校准日，72种instrument
Quantlib_price_zerorate=np.full((62,72),1.0)

# ZeroBond结果的初始化
ZeroBond_Quantlib_Model_Price=np.full((62,72),1.0)
ZeroBond_Quantlib_Market_Price=np.full((62,72),1.0)
ZeroBond_Quantlib_Implied_Vol=np.full((62,72),1.0)
ZeroBond_Quantlib_Market_Vol=np.full((62,72),1.0)
ZeroBond_Quantlib_Rel_Error_Price=np.full((62,72),1.0)
ZeroBond_Quantlib_Rel_Error_vol=np.full((62,72),1.0)

# Swaption结果的初始化
Swaption_Quantlib_Model_Price=np.full((62,72),1.0)
Swaption_Quantlib_Market_Price=np.full((62,72),1.0)
Swaption_Quantlib_Implied_Vol=np.full((62,72),1.0)
Swaption_Quantlib_Market_Vol=np.full((62,72),1.0)
Swaption_Quantlib_Rel_Error_Price=np.full((62,72),1.0)
Swaption_Quantlib_Rel_Error_vol=np.full((62,72),1.0)

In [ ]:
today=ql.Date(28,4,2023)
for num_day in range(62):
    if today.weekday() == ql.Saturday:
        today += ql.Period(2, ql.Days)  # 如果是周六，跳过周末，将日期调整到下一个周一
    elif today.weekday() == ql.Sunday:
        today += ql.Period(1, ql.Days)  # 如果是周日，将日期调整到下一个周一
        
    
    
    #生成TermStructure
    curve_start_date = today
    curve_date = [curve_start_date]
    current_date = curve_start_date
    while len(curve_date) < 601:
        current_date = ql.Date(current_date.dayOfMonth(), current_date.month(), current_date.year())
        current_date += ql.Period(1, ql.Months)
        curve_date.append(current_date)
    #len(curve_date)=601
        
    ql.Settings.instance().evaluationDate = today   #evaluationDate
    term_structure = ql.YieldTermStructureHandle(ql.ZeroCurve(curve_date, ZeroRate[num_day,:], ql.Actual360()))
    index = ql.USDLibor(ql.Period('6M'),term_structure)
    
    
    
    CalibrationData = namedtuple("CalibrationData","start, length, volatility")
    data=[]
    for instru_type in range(0,72):
        calibration_data = CalibrationData(Maturity[instru_type], Tenor[instru_type], using_swaption_vol[num_day,0] )
        data.append(calibration_data)
        
        
    model = ql.HullWhite(term_structure)
    engine = ql.JamshidianSwaptionEngine(model)
    swaptions = create_swaption_helpers(data, index, term_structure, engine)
    optimization_method = ql.LevenbergMarquardt(1.0e-8,1.0e-8,1.0e-8)
    end_criteria = ql.EndCriteria(10000, 100, 1e-6, 1e-8, 1e-8)
    model.calibrate(swaptions, optimization_method, end_criteria)
    a, sigma = model.params()
    
    
    
    result_save=calibration_report(swaptions, data)
    Swaption_Quantlib_Model_Price[num_day,:]=result_save['Model Price'].values
    Swaption_Quantlib_Market_Price[num_day,:]=result_save['Market Price'].values
    Swaption_Quantlib_Implied_Vol[num_day,:]=result_save['Implied Vol'].values
    Swaption_Quantlib_Market_Vol[num_day,:]=result_save['Market Vol'].values
    Swaption_Quantlib_Rel_Error_Price[num_day,:]=result_save['Rel Error Price'].values
    Swaption_Quantlib_Rel_Error_vol[num_day,:]=result_save['Rel Error Vols'].values
        
    today += ql.Period(1, ql.Days)
    